In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# System
import os
import sys
sys.path.append('/home/helfrech/Tools/Toolbox/utils')

# Maths
import numpy as np

# Plotting
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

# ML
from regression import PCovR, KPCovR, SparseKPCovR
from regression import LR, KRR
from kernels import build_kernel, linear_kernel, gaussian_kernel
from kernels import center_kernel, center_kernel_fast
from kernels import center_kernel_oos, center_kernel_oos_fast
from soap import extract_species_pair_groups
from errors import MAE

from sklearn.svm import SVC, LinearSVC
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import LogisticRegression

# Utilities
import h5py
import json
import itertools
from tqdm.notebook import tqdm
import project_utils as utils
from tools import load_json

# Import COSMO style toolkit
import cosmoplot.colorbars as cosmocbars
import cosmoplot.utils as cosmoutils
import cosmoplot.style as cosmostyle

cosmostyle.set_style('article')
colorList = cosmostyle.color_cycle

In /home/helfrech/.config/matplotlib/stylelib/cosmo.mplstyle: 
The savefig.frameon rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In /home/helfrech/.config/matplotlib/stylelib/cosmoLarge.mplstyle: 
The savefig.frameon rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
/home/helfrech/ENVIRONMENTS/ZEOLITES/lib/python3.6/_collections_abc.py:841: MatplotlibDeprecationWarning: 
The savefig.frameon rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
  self[key] = other[key]


In [3]:
# sys.path.append('/scratch/helfrech/Sync/GDrive/Projects/KPCovR/kernel-tutorials')
# sys.path.append('/scratch/helfrech/Sync/GDrive/Projects/KPCovR/KernelPCovR/analysis/scripts')
# from utilities.sklearn_covr.kpcovr import KernelPCovR as KPCovR2
# from utilities.sklearn_covr.pcovr import PCovR as PCovR2
# from helpers import l_regr, l_kpcovr

# Load train and test splits

In [4]:
# Load SOAP cutoffs
with open('../Processed_Data/soap_hyperparameters.json', 'r') as f:
    soap_hyperparameters = json.load(f)
    
cutoffs = soap_hyperparameters['interaction_cutoff']

In [5]:
# Load train and test set indices for Deem
idxs_deem_train = np.loadtxt('../Processed_Data/DEEM_10k/train.idxs', dtype=int)
idxs_deem_test = np.loadtxt('../Processed_Data/DEEM_10k/test.idxs', dtype=int)

# Total number of structures
n_deem_train = idxs_deem_train.size
n_deem_test = idxs_deem_test.size
n_deem = n_deem_train + n_deem_test

print(n_deem_train, n_deem_test)

7750 2250


In [6]:
# Make dummy DEEM cantons
cantons_deem = np.ones(n_deem, dtype=int) * 4

In [7]:
# Load IZA cantons
cantons_iza = np.loadtxt('../Raw_Data/GULP/IZA_226/cantons.txt', usecols=1, dtype=int)
RWY = np.nonzero(cantons_iza == 4)[0][0]

In [8]:
cantons_iza = np.delete(cantons_iza, RWY)
n_iza = len(cantons_iza)

In [9]:
idxs_iza_train_file = '../Processed_Data/IZA_226/train.idxs' 
idxs_iza_test_file = '../Processed_Data/IZA_226/test.idxs'

# Load IZA train and test set indices
try:
    idxs_iza_train = np.loadtxt(idxs_iza_train_file, dtype=int)
    idxs_iza_test = np.loadtxt(idxs_iza_test_file, dtype=int)
    n_iza_train = len(idxs_iza_train)
    n_iza_test = len(idxs_iza_test)
    
    print(n_iza_train, n_iza_test)

# Compute indices if they don't exist
except IOError:

    # Select IZA sample
    n_iza_train = n_iza // 2
    n_iza_test = n_iza - n_iza_train
    idxs_iza = np.arange(0, n_iza)
    np.random.shuffle(idxs_iza)

    idxs_iza_train = idxs_iza[0:n_iza_train]
    idxs_iza_test = idxs_iza[n_iza_train:]
    
    np.savetxt(idxs_iza_train_file, idxs_iza_train, fmt='%d')
    np.savetxt(idxs_iza_test_file, idxs_iza_test, fmt='%d')

112 113


In [10]:
# Build set of "master" canton labels
cantons_train = {}
cantons_test = {}

cantons_train[4] = np.concatenate((cantons_iza[idxs_iza_train], cantons_deem[idxs_deem_train]))
cantons_test[4] = np.concatenate((cantons_iza[idxs_iza_test], cantons_deem[idxs_deem_test]))

cantons_train[2] = np.concatenate((np.ones(len(idxs_iza_train), dtype=int),
                                   np.ones(len(idxs_deem_train), dtype=int) * 2))
cantons_test[2] = np.concatenate((np.ones(len(idxs_iza_test), dtype=int),
                                  np.ones(len(idxs_deem_test), dtype=int) * 2))

# Model setup

In [11]:
model_dir = '../Processed_Data/Models'

In [12]:
class_names = {}
class_names[2] = ['IZA', 'DEEM']
class_names[4] = ['IZA1', 'IZA2', 'IZA3', 'DEEM']

In [13]:
# Global model parameters
# TODO: or use .get_params()?
# TODO: load from optimization?
# TODO: should we make sure break_ties=True?

# NOTE: with balance weights, the sum of the weights times class frequencies is one
# NOTE: LSVC decision functions are OVR (or Crammer-Singer) only; KSVC decision functions
# are always computed with OVO, but can be converted to OVR. Hence we use OVR with both
# LSVC and KSVC for consistency
svc_kwargs = dict(linear=dict(penalty='l2',
                              loss='squared_hinge',
                              dual=False,
                              multi_class='ovr',
                              #class_weight=None,
                              class_weight='balanced',
                              fit_intercept=True,
                              intercept_scaling=1.0,
                              tol=1.0E-3,
                              max_iter=5000,
                              C=1.0),
                  kernel=dict(kernel='precomputed',
                              decision_function_shape='ovr',
                              #class_weight=None,
                              class_weight='balanced',
                              break_ties=False,
                              tol=1.0E-3,
                              max_iter=-1,
                              C=10.0))

pcovr_kwargs = dict(linear=dict(n_components=None, alpha=0.0, regularization=1.0E-12),
                    kernel=dict(n_components=None, alpha=0.0, regularization=1.0E-12))

In [14]:
# Slices for saving KSVC and KPCovR outputs
deem_train_slice = slice(n_iza_train, None)
deem_test_slice = slice(n_iza_test, None)
iza_train_slice = slice(0, n_iza_train)
iza_test_slice = slice(0, n_iza_test)

# Build kernels

In [15]:
# Load the kernels or compute if they don't exist
for cutoff in cutoffs:
    for kernel_type in ('linear', 'gaussian'):
        kernel_name = kernel_type.capitalize()
        
        work_dir = f'{model_dir}/{cutoff}/Kernel_Models/{kernel_name}/KSVC-KPCovR'
        
        if not os.path.exists(work_dir):
            os.makedirs(work_dir)
        
        # File to store kernels for re-use
        kernel_file = f'{work_dir}/structure_kernels.hdf5'
        kernel_parameter_file = f'{work_dir}/volumes_mae_parameters.json'

        if not os.path.exists(kernel_file):
            
            # SOAP files (atomwise, FPS'ed features)
            deem_file = f'{deem_dir}/{cutoff}/soaps.hdf5'
            iza_file = f'{iza_dir}/{cutoff}/soaps.hdf5'

            # Assemble the train and test set SOAPs from IZA and DEEM
            soaps_train, soaps_test = utils.load_soaps(deem_file, iza_file,
                                                       idxs_deem_train, idxs_deem_test,
                                                       idxs_iza_train, idxs_iza_test,
                                                       idxs_iza_delete=[RWY])

            # Compute kernels
            # This can be consolidated if doing linear KRR
            if kernel_type == 'gaussian':
                kernel_parameters = load_json(kernel_parameter_file)
                kernel_parameters.pop('sigma')
                kernel_parameters.pop('regularization')
            else:
                kernel_parameters = dict(kernel='linear', zeta=1)
            
            utils.compute_kernels(soaps_train, soaps_test, **kernel_parameters, kernel_file=kernel_file)

# Kernel models

In [16]:
deem_name = 'DEEM_10k'
iza_name = 'IZA_226onDEEM_10k'
deem_dir = f'../Processed_Data/{deem_name}/Data'
iza_dir = f'../Processed_Data/{iza_name}/Data'

## KernelSVC

In [17]:
for cutoff in tqdm(cutoffs, desc='Cutoff', leave=True):
    
    for kernel_type in tqdm(('linear', 'gaussian'), desc='Kernel', leave=False):
        kernel_name = kernel_type.capitalize()
        
        kernel_dir = f'{model_dir}/{cutoff}/Kernel_Models/{kernel_name}/KSVC-KPCovR'
        
        # File to store kernels for re-use
        kernel_file = f'{kernel_dir}/structure_kernels.hdf5'
        
        # Load kernels
        K_train, K_test, K_test_test = utils.load_kernels(kernel_file)

        # Center and scale kernels
        K_train, K_test, K_test_test = \
            utils.preprocess_kernels(K_train, K_test=K_test, 
                                     K_test_test=K_test_test, K_bridge=K_test)
        
        for n_cantons in tqdm((2, 4), desc='Classes', leave=False):
            
            # Prepare outputs
            output_dir = f'Kernel_Models/{kernel_name}/KSVC-KPCovR/{n_cantons}-Class'
            
            if not os.path.exists(f'{deem_dir}/{cutoff}/{output_dir}'):
                os.makedirs(f'{deem_dir}/{cutoff}/{output_dir}')
                
            if not os.path.exists(f'{iza_dir}/{cutoff}/{output_dir}'):
                os.makedirs(f'{iza_dir}/{cutoff}/{output_dir}')
            
            svc_df_deem_file = f'{deem_dir}/{cutoff}/{output_dir}/svc_structure_dfs.dat'
            svc_df_iza_file = f'{iza_dir}/{cutoff}/{output_dir}/svc_structure_dfs.dat'
            
            svc_cantons_deem_file = f'{deem_dir}/{cutoff}/{output_dir}/svc_structure_cantons.dat'
            svc_cantons_iza_file = f'{iza_dir}/{cutoff}/{output_dir}/svc_structure_cantons.dat'
            
            parameter_dir = f'{kernel_dir}/{n_cantons}-Class'
            svc_parameter_file = f'{parameter_dir}/svc_parameters.json'

            # Run KSVC
            svc_parameters = svc_kwargs['kernel'].copy() #load_json(svc_parameter_file) ###
            df_train, df_test, predicted_cantons_train, predicted_cantons_test = \
                utils.do_svc(K_train, K_test, cantons_train[n_cantons], cantons_test[n_cantons], 
                             svc_type='kernel', **svc_parameters,
                             outputs=['decision_functions', 'predictions'])
            
            # Save IZA and DEEM KSVC decision functions
            utils.split_and_save(df_train, df_test,
                           idxs_deem_train, idxs_deem_test,
                           deem_train_slice, deem_test_slice,
                           output=svc_df_deem_file, output_format='%f',
                           hdf5_attrs=None)                           
            
            utils.split_and_save(df_train, df_test,
                                 idxs_iza_train, idxs_iza_test,
                                 iza_train_slice, iza_test_slice,
                                 output=svc_df_iza_file, output_format='%f',
                                 hdf5_attrs=None)
            
            # Save IZA and DEEM KSVC canton predictions
            utils.split_and_save(predicted_cantons_train, predicted_cantons_test,
                                 idxs_deem_train, idxs_deem_test,
                                 deem_train_slice, deem_test_slice,
                                 output=svc_cantons_deem_file, output_format='%d',
                                 hdf5_attrs=None)
            
            utils.split_and_save(predicted_cantons_train, predicted_cantons_test,
                                 idxs_iza_train, idxs_iza_test,
                                 iza_train_slice, iza_test_slice,
                                 output=svc_cantons_iza_file, output_format='%d',
                                 hdf5_attrs=None)

## KRR check

In [18]:
for cutoff in tqdm(cutoffs, desc='Cutoff', leave=True):
    
    for kernel_type in tqdm(('linear', 'gaussian'), desc='Kernel', leave=False):
        kernel_name = kernel_type.capitalize()
        
        kernel_dir = f'{model_dir}/{cutoff}/Kernel_Models/{kernel_name}/KSVC-KPCovR'
        
        # Load kernels
        kernel_file = f'{kernel_dir}/structure_kernels.hdf5'
        K_train, K_test, K_test_test = utils.load_kernels(kernel_file)

        # Center and scale kernels
        K_train, K_test, K_test_test = \
            utils.preprocess_kernels(K_train, K_test=K_test, 
                                     K_test_test=K_test_test, K_bridge=K_test)
        
        for n_cantons in tqdm((2, 4), desc='Classes', leave=False):
            
            # Load decision functions
            input_dir = f'Kernel_Models/{kernel_name}/KSVC-KPCovR/{n_cantons}-Class'
            svc_df_deem_file = f'{deem_dir}/{cutoff}/{input_dir}/svc_structure_dfs.dat'
            svc_df_iza_file = f'{iza_dir}/{cutoff}/{input_dir}/svc_structure_dfs.dat'
            
            df_train, df_test = utils.load_data(svc_df_deem_file, svc_df_iza_file,
                                                idxs_deem_train, idxs_deem_test,
                                                idxs_iza_train, idxs_iza_test)
            
            # Center and scale the decision functions
            df_train, df_test, df_center, df_scale = \
                utils.preprocess_data(df_train, df_test)

            # Check that KRR can reproduce the decision functions
            dfp_train, dfp_test = utils.regression_check(K_train, K_test, 
                                                         df_train, df_test, 
                                                         regression_type='kernel')
            
            print(MAE(dfp_train, df_train))
            print(MAE(dfp_test, df_test))

7.240392803935956e-08
7.73491686113738e-08
[0.24189382 0.22251397 0.22835672 0.24221625]
[0.27390314 0.23967644 0.24264844 0.29071136]


2.9459469516717804e-09
5.900088990810978e-07
[0.00712177 0.0055098  0.00742347 0.01373742]
[1.68196984 1.30112342 1.76510013 3.06066357]


1.2979401409084714e-07
1.4007314250003076e-07
[0.20642866 0.19691288 0.19903367 0.20223848]
[0.22979768 0.20962861 0.21623932 0.27074206]


1.0534641950847684e-12
3.1811331548687326e-07
[3.64314101e-11 4.93698991e-11 6.29833637e-11 3.77345088e-11]
[0.24387092 0.24361182 0.28670495 0.23501228]



## KPCovR

In [19]:
for cutoff in tqdm(cutoffs, desc='Cutoff', leave=True):
    
    for kernel_type in tqdm(('linear', 'gaussian'), desc='Kernel', leave=False):
        kernel_name = kernel_type.capitalize()
        
        kernel_dir = f'{model_dir}/{cutoff}/Kernel_Models/{kernel_name}/KSVC-KPCovR'
        
        # Load kernels
        kernel_file = f'{kernel_dir}/structure_kernels.hdf5'
        K_train, K_test, K_test_test = utils.load_kernels(kernel_file)

        # Center and scale kernels
        K_train, K_test, K_test_test = \
            utils.preprocess_kernels(K_train, K_test=K_test, 
                                     K_test_test=K_test_test, K_bridge=K_test)
        
        for n_cantons in tqdm((2, 4), desc='Classes', leave=False):
            
            # Prepare outputs
            output_dir = f'Kernel_Models/{kernel_name}/KSVC-KPCovR/{n_cantons}-Class'
            
            svc_df_deem_file = f'{deem_dir}/{cutoff}/{output_dir}/svc_structure_dfs.dat'
            svc_df_iza_file = f'{iza_dir}/{cutoff}/{output_dir}/svc_structure_dfs.dat'
            
            pcovr_projection_deem_file = f'{deem_dir}/{cutoff}/{output_dir}/pcovr_structures.hdf5' 
            pcovr_projection_iza_file = f'{iza_dir}/{cutoff}/{output_dir}/pcovr_structures.hdf5'
            
            pcovr_df_deem_file = f'{deem_dir}/{cutoff}/{output_dir}/pcovr_structure_dfs.dat'
            pcovr_df_iza_file = f'{iza_dir}/{cutoff}/{output_dir}/pcovr_structure_dfs.dat'
            
            pcovr_cantons_deem_file = f'{deem_dir}/{cutoff}/{output_dir}/pcovr_structure_cantons.dat'
            pcovr_cantons_iza_file = f'{iza_dir}/{cutoff}/{output_dir}/pcovr_structure_cantons.dat'
            
            parameter_dir = f'{kernel_dir}/{n_cantons}-Class'
            pcovr_parameter_file = f'{parameter_dir}/pcovr_parameters.json'
            svc_parameter_file = f'{parameter_dir}/svc_parameters.json'
            
            df_train, df_test = utils.load_data(svc_df_deem_file, svc_df_iza_file,
                                                idxs_deem_train, idxs_deem_test,
                                                idxs_iza_train, idxs_iza_test)
            
            # Center and scale the decision functions
            df_train, df_test, df_center, df_scale = \
                utils.preprocess_data(df_train, df_test)
            
            # Run KPCovR
            pcovr_parameters = pcovr_kwargs['kernel'].copy() #load_json(pcovr_parameter_file) ###
            svc_parameters = svc_kwargs['kernel'].copy() #load_json(svc_parameter_file) ###
            T_train, T_test, dfp_train, dfp_test = \
                utils.do_pcovr(K_train, K_test, df_train, df_test, 
                               pcovr_type='kernel', **pcovr_parameters)
                      
            # Post process the KPCovR decision functions
            # (i.e., turn them back into canton predictions)
            predicted_cantons_train, predicted_cantons_test = \
                utils.postprocess_decision_functions(dfp_train, dfp_test, df_center, df_scale,
                                                     df_type=svc_parameters['decision_function_shape'],
                                                     n_classes=n_cantons)
            
            # Save IZA and DEEM KPCovR projections
            utils.split_and_save(T_train, T_test,
                                 idxs_deem_train, idxs_deem_test,
                                 deem_train_slice, deem_test_slice,
                                 output=pcovr_projection_deem_file, output_format='%f',
                                 hdf5_attrs=pcovr_parameters)
            
            utils.split_and_save(T_train, T_test,
                                 idxs_iza_train, idxs_iza_test,
                                 iza_train_slice, iza_test_slice,
                                 output=pcovr_projection_iza_file, output_format='%f',
                                 hdf5_attrs=pcovr_parameters)            
            
            # Save IZA and DEEM KPCovR decision functions
            utils.split_and_save(dfp_train, dfp_test,
                                 idxs_deem_train, idxs_deem_test,
                                 deem_train_slice, deem_test_slice,
                                 output=pcovr_df_deem_file, output_format='%f',
                                 hdf5_attrs=None)
            
            utils.split_and_save(dfp_train, dfp_test,
                                 idxs_iza_train, idxs_iza_test,
                                 iza_train_slice, iza_test_slice,
                                 output=pcovr_df_iza_file, output_format='%f', 
                                 hdf5_attrs=None)
            
            # Save IZA and DEEM KPCovR canton predictions
            utils.split_and_save(predicted_cantons_train, predicted_cantons_test,
                                 idxs_deem_train, idxs_deem_test,
                                 deem_train_slice, deem_test_slice,
                                 output=pcovr_cantons_deem_file, output_format='%d',
                                 hdf5_attrs=None)
            
            utils.split_and_save(predicted_cantons_train, predicted_cantons_test,
                                 idxs_iza_train, idxs_iza_test,
                                 iza_train_slice, iza_test_slice,
                                 output=pcovr_cantons_iza_file, output_format='%d',
                                 hdf5_attrs=None)

# Linear Models

In [20]:
# Linear model setup
n_species = 2
group_names = {'power': ['OO', 'OSi', 'SiSi', 
                         'OO+OSi', 'OO+SiSi', 'OSi+SiSi',
                         'OO+OSi+SiSi'], 
               'radial': ['O', 'Si', 'O+Si']}

In [21]:
deem_name = 'DEEM_10k'
iza_name = 'IZA_226'
deem_dir = f'../Processed_Data/{deem_name}/Data'
iza_dir = f'../Processed_Data/{iza_name}/Data'

## LinearSVC

In [22]:
for cutoff in tqdm(cutoffs, desc='Cutoff', leave=True):
    linear_dir = f'{model_dir}/{cutoff}/Linear_Models/LSVC-LPCovR'
    
    for spectrum_type in tqdm(('power', 'radial'), desc='Spectrum', leave=False):
        spectrum_name = spectrum_type.capitalize()
        
        # Load SOAPs
        deem_file = f'{deem_dir}/{cutoff}/soaps_{spectrum_type}_full_avg_nonorm.hdf5'
        iza_file = f'{iza_dir}/{cutoff}/soaps_{spectrum_type}_full_avg_nonorm.hdf5'
        
        soaps_train, soaps_test = utils.load_soaps(deem_file, iza_file,
                                                   idxs_deem_train, idxs_deem_test,
                                                   idxs_iza_train, idxs_iza_test,
                                                   idxs_iza_delete=[RWY],
                                                   train_test_concatenate=True)

        # Scale the SOAPs so they are of a 'usable' magnitude for the SVC
        soaps_train, soaps_test = utils.preprocess_soaps(soaps_train, soaps_test)
        
        n_features = soaps_train.shape[1]
        feature_groups = extract_species_pair_groups(n_features, n_species, 
                                                     spectrum_type=spectrum_type,
                                                     combinations=True)

        for species_pairing, feature_idxs in zip(tqdm(group_names[spectrum_type], 
                                                      desc='Species', leave=False),
                                                 feature_groups):
            
            for n_cantons in tqdm((2, 4), desc='Classes', leave=False):
                                
                # Prepare outputs
                output_dir = f'Linear_Models/LSVC-LPCovR/{n_cantons}-Class/{spectrum_name}/{species_pairing}'
                
                if not os.path.exists(f'{deem_dir}/{cutoff}/{output_dir}'):
                    os.makedirs(f'{deem_dir}/{cutoff}/{output_dir}')
                    
                if not os.path.exists(f'{iza_dir}/{cutoff}/{output_dir}'):
                    os.makedirs(f'{iza_dir}/{cutoff}/{output_dir}')
                
                svc_df_deem_file = f'{deem_dir}/{cutoff}/{output_dir}/svc_structure_dfs.dat'
                svc_df_iza_file = f'{iza_dir}/{cutoff}/{output_dir}/svc_structure_dfs.dat'

                svc_cantons_deem_file = f'{deem_dir}/{cutoff}/{output_dir}/svc_structure_cantons.dat'
                svc_cantons_iza_file = f'{iza_dir}/{cutoff}/{output_dir}/svc_structure_cantons.dat'
                                
                parameter_dir = f'{linear_dir}/{n_cantons}-Class/{spectrum_name}/{species_pairing}'
                svc_parameter_file = f'{parameter_dir}/svc_parameters.json'
                
                svc_weights_file = f'{parameter_dir}/svc_weights.dat'

                # Run LSVC
                svc_parameters = svc_kwargs['linear'].copy() #load_json(svc_parameter_file) ###
                df_train, df_test, predicted_cantons_train, predicted_cantons_test, weights = \
                    utils.do_svc(soaps_train[:, feature_idxs], soaps_test[:, feature_idxs], 
                                 cantons_train[n_cantons], cantons_test[n_cantons], 
                                 svc_type='linear', **svc_parameters,
                                 outputs=['decision_functions', 'predictions', 'weights'])

                # Save IZA and DEEM LSVC decision functions
                utils.split_and_save(df_train, df_test,
                                     idxs_deem_train, idxs_deem_test,
                                     deem_train_slice, deem_test_slice,
                                     output=svc_df_deem_file, output_format='%f',
                                     hdf5_attrs=None)
                
                utils.split_and_save(df_train, df_test,
                                     idxs_iza_train, idxs_iza_test,
                                     iza_train_slice, iza_test_slice,
                                     output=svc_df_iza_file, output_format='%f',
                                     hdf5_attrs=None)

                # Save IZA and DEEM LSVC canton predictions
                utils.split_and_save(predicted_cantons_train, predicted_cantons_test,
                                     idxs_deem_train, idxs_deem_test,
                                     deem_train_slice, deem_test_slice,
                                     output=svc_cantons_deem_file, output_format='%d',
                                     hdf5_attrs=None)
                
                utils.split_and_save(predicted_cantons_train, predicted_cantons_test,
                                     idxs_iza_train, idxs_iza_test,
                                     iza_train_slice, iza_test_slice,
                                     output=svc_cantons_iza_file, output_format='%d',
                                     hdf5_attrs=None)
                
                # Save weights
                np.savetxt(svc_weights_file, weights)

## LR check

In [23]:
for cutoff in tqdm(cutoffs, desc='Cutoff', leave=True):
    
    for spectrum_type in tqdm(('power', 'radial'), desc='Spectrum', leave=False):
        spectrum_name = spectrum_type.capitalize()
        
        # Load SOAPs
        deem_file = f'{deem_dir}/{cutoff}/soaps_{spectrum_type}_full_avg_nonorm.hdf5'
        iza_file = f'{iza_dir}/{cutoff}/soaps_{spectrum_type}_full_avg_nonorm.hdf5'

        soaps_train, soaps_test = utils.load_soaps(deem_file, iza_file,
                                                   idxs_deem_train, idxs_deem_test,
                                                   idxs_iza_train, idxs_iza_test,
                                                   idxs_iza_delete=[RWY],
                                                   train_test_concatenate=True)
        
        n_features = soaps_train.shape[1]
        feature_groups = extract_species_pair_groups(n_features, n_species, 
                                                     spectrum_type=spectrum_type, 
                                                     combinations=True)
        
        for species_pairing, feature_idxs in zip(tqdm(group_names[spectrum_type], 
                                                      desc='Species', leave=False),
                                                 feature_groups):
            
            x_train = soaps_train[:, feature_idxs]
            x_test = soaps_test[:, feature_idxs]

            # Preprocess the SOAPs like the decision functions
            # (i.e., center and scale) for the regression.
            x_train, x_test, x_center, x_scale = \
                utils.preprocess_data(x_train, x_test)
            
            for n_cantons in tqdm((2, 4), desc='Classes', leave=False):
                
                # Load decision functions
                input_dir = f'Linear_Models/LSVC-LPCovR/{n_cantons}-Class/{spectrum_name}/{species_pairing}'
                svc_df_deem_file = f'{deem_dir}/{cutoff}/{input_dir}/svc_structure_dfs.dat'
                svc_df_iza_file = f'{iza_dir}/{cutoff}/{input_dir}/svc_structure_dfs.dat'

                df_train, df_test = utils.load_data(svc_df_deem_file, svc_df_iza_file,
                                                    idxs_deem_train, idxs_deem_test,
                                                    idxs_iza_train, idxs_iza_test)
                
                # Center and scale the decision functions
                df_train, df_test, df_center, df_scale = \
                    utils.preprocess_data(df_train, df_test)
                
                # Check that LR can reproduce the decision functions
                dfp_train, dfp_test = utils.regression_check(x_train, x_test, 
                                                             df_train, df_test, 
                                                             regression_type='linear')
                
                print(MAE(dfp_train, df_train))
                print(MAE(dfp_test, df_test))

5.951420625549286e-07
1.6733860364302191e-06
[1.27004889e-07 1.59816071e-07 3.71299635e-07 4.15208233e-07]
[3.51954034e-07 4.60954017e-07 9.21982539e-07 1.21917692e-06]


1.1336435050418972e-06
2.2915594072670793e-06
[2.78726764e-07 3.50718283e-07 6.09254550e-07 5.52089460e-07]
[6.43106398e-07 6.41818354e-07 1.27383805e-06 1.11330573e-06]


1.401468187425319e-06
1.7834418977920897e-06
[3.16946628e-07 4.13547584e-07 1.70760246e-06 4.12898734e-07]
[3.77048792e-07 4.92051099e-07 2.19583235e-06 4.94628453e-07]


1.8200318111673028e-06
6.147678787394855e-06
[6.11355423e-07 6.00204629e-07 8.65233702e-07 8.65623998e-07]
[2.26922278e-06 1.92278123e-06 4.01612505e-06 2.51576042e-06]


1.3546632619055442e-06
2.766828475847418e-06
[2.53812100e-07 4.11765710e-07 8.53094381e-07 5.09669025e-07]
[8.12143717e-07 1.01092561e-06 1.79882459e-06 1.34374931e-06]


...

1.830049615923203e-06
2.7324137449766664e-06
[1.35509822e-06 1.22292678e-06 9.31614467e-07 8.92945925e-07]
[2.06130941e-06 1.89368518e-06 1.42331693e-06 1.33446588e-06]


1.861334650866247e-06
3.3297284218006098e-06
[1.14042985e-06 8.98084707e-07 7.32659623e-07 8.90690077e-07]
[2.11802873e-06 1.77845048e-06 1.35734849e-06 1.62056273e-06]


2.5441362296163235e-07
2.5001778686504963e-07
[1.03283577e-07 8.66931567e-08 1.66168109e-07 1.50184284e-07]
[1.03222293e-07 8.61848068e-08 1.63436131e-07 1.48061277e-07]


3.3018848518137207e-07
3.3124577381153986e-07
[1.79514578e-07 1.45543356e-07 2.05262148e-07 3.83007699e-07]
[1.75973643e-07 1.45824762e-07 2.05914799e-07 3.88154357e-07]


2.4769552778583826e-07
2.46530652628551e-07
[9.31849887e-08 8.72353987e-08 1.45886103e-07 1.49308888e-07]
[9.21033395e-08 8.95295424e-08 1.45161181e-07 1.47399920e-07]



## PCovR

In [24]:
for cutoff in tqdm(cutoffs, desc='Cutoff', leave=True):
    linear_dir = f'{model_dir}/{cutoff}/Linear_Models/LSVC-LPCovR'
    
    for spectrum_type in tqdm(('power', 'radial'), desc='Spectrum', leave=False):
        spectrum_name = spectrum_type.capitalize()
        
        # Load SOAPs
        deem_file = f'{deem_dir}/{cutoff}/soaps_{spectrum_type}_full_avg_nonorm.hdf5'
        iza_file = f'{iza_dir}/{cutoff}/soaps_{spectrum_type}_full_avg_nonorm.hdf5'

        soaps_train, soaps_test = utils.load_soaps(deem_file, iza_file,
                                                   idxs_deem_train, idxs_deem_test,
                                                   idxs_iza_train, idxs_iza_test,
                                                   idxs_iza_delete=[RWY],
                                                   train_test_concatenate=True)
        
        # Scale the SOAPs so they are of a 'usable' magnitude for the SVC
        #soaps_train, soaps_test = utils.preprocess_soaps(soaps_train, soaps_test)
        
        n_features = soaps_train.shape[1]
        feature_groups = extract_species_pair_groups(n_features, n_species, 
                                                     spectrum_type=spectrum_type,
                                                     combinations=True)
        
        for species_pairing, feature_idxs in zip(tqdm(group_names[spectrum_type], 
                                                      desc='Species', leave=False),
                                                 feature_groups):
            
            x_train = soaps_train[:, feature_idxs]
            x_test = soaps_test[:, feature_idxs]

            # Preprocess the SOAPs like the decision functions
            # (i.e., center and scale) for the regression.
            x_train, x_test, x_center, x_scale = \
                utils.preprocess_data(x_train, x_test)
            
            for n_cantons in tqdm((2, 4), desc='Classes', leave=False):
                
                # Prepare outputs
                output_dir = f'Linear_Models/LSVC-LPCovR/{n_cantons}-Class/{spectrum_name}/{species_pairing}'
                
                svc_df_deem_file = f'{deem_dir}/{cutoff}/{output_dir}/svc_structure_dfs.dat'
                svc_df_iza_file = f'{iza_dir}/{cutoff}/{output_dir}/svc_structure_dfs.dat'
                
                pcovr_projection_deem_file = f'{deem_dir}/{cutoff}/{output_dir}/pcovr_structures.hdf5'
                pcovr_projection_iza_file = f'{iza_dir}/{cutoff}/{output_dir}/pcovr_structures.hdf5'
                
                pcovr_df_deem_file = f'{deem_dir}/{cutoff}/{output_dir}/pcovr_structure_dfs.dat'
                pcovr_df_iza_file = f'{iza_dir}/{cutoff}/{output_dir}/pcovr_structure_dfs.dat'
                
                pcovr_cantons_deem_file = f'{deem_dir}/{cutoff}/{output_dir}/pcovr_structure_cantons.dat'
                pcovr_cantons_iza_file = f'{iza_dir}/{cutoff}/{output_dir}/pcovr_structure_cantons.dat'
                
                parameter_dir = f'{linear_dir}/{n_cantons}-Class/{spectrum_name}/{species_pairing}'
                pcovr_parameter_file = f'{parameter_dir}/pcovr_parameters.json'
                
                df_train, df_test = utils.load_data(svc_df_deem_file, svc_df_iza_file,
                                                    idxs_deem_train, idxs_deem_test,
                                                    idxs_iza_train, idxs_iza_test)
            
                # Center and scale the decision functions
                df_train, df_test, df_center, df_scale = \
                    utils.preprocess_data(df_train, df_test)
                
                # Run PCovR
                pcovr_parameters = pcovr_kwargs['linear'].copy() #load_json(pcovr_parameter_file) ###
                svc_parameters = svc_kwargs['linear'].copy() #load_json(svc_parameter_file) ###
                T_train, T_test, dfp_train, dfp_test = \
                    utils.do_pcovr(x_train, x_test, df_train, df_test, 
                                   pcovr_type='linear', **pcovr_parameters)

                # Post process the PCovR decision functions
                # (i.e., turn them back into canton predictions)
                predicted_cantons_train, predicted_cantons_test = \
                    utils.postprocess_decision_functions(dfp_train, dfp_test, df_center, df_scale,
                                                         df_type=svc_parameters['multi_class'],
                                                         n_classes=n_cantons)
                
                # Save IZA and DEEM PCovR projections
                utils.split_and_save(T_train, T_test,
                                     idxs_deem_train, idxs_deem_test,
                                     deem_train_slice, deem_test_slice,
                                     output=pcovr_projection_deem_file, output_format='%f',
                                     hdf5_attrs=pcovr_parameters)
                
                utils.split_and_save(T_train, T_test,
                                     idxs_iza_train, idxs_iza_test,
                                     iza_train_slice, iza_test_slice,
                                     output=pcovr_projection_iza_file, output_format='%f',
                                     hdf5_attrs=pcovr_parameters)

                # Save IZA and DEEM PCovR decision functions
                utils.split_and_save(dfp_train, dfp_test,
                                     idxs_deem_train, idxs_deem_test,
                                     deem_train_slice, deem_test_slice,
                                     output=pcovr_df_deem_file, output_format='%f',
                                     hdf5_attrs=None)
                
                utils.split_and_save(dfp_train, dfp_test,
                                     idxs_iza_train, idxs_iza_test,
                                     iza_train_slice, iza_test_slice,
                                     output=pcovr_df_iza_file, output_format='%f',
                                     hdf5_attrs=None)

                # Save IZA and DEEM PCovR canton predictions
                utils.split_and_save(predicted_cantons_train, predicted_cantons_test,
                                     idxs_deem_train, idxs_deem_test,
                                     deem_train_slice, deem_test_slice,
                                     output=pcovr_cantons_deem_file, output_format='%d',
                                     hdf5_attrs=None)
                
                utils.split_and_save(predicted_cantons_train, predicted_cantons_test,
                                     idxs_iza_train, idxs_iza_test,
                                     iza_train_slice, iza_test_slice,
                                     output=pcovr_cantons_iza_file, output_format='%d',
                                     hdf5_attrs=None)

# Logistic Regression